# The purpose of this notebook

This is the final notebook that makes figures for the paper.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
n_boot = 500

In [ ]:
from IPython.display import display, HTML

import string

import numpy as np
np.set_printoptions(precision=3)

import pandas as pd
pd.set_option('display.max_rows', 500)

import scipy.stats as st
import statsmodels.stats.multitest as mt

import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import seaborn as sns

import suppression as s
import utils

In [ ]:
pd.__version__

In [ ]:
sns.__version__

In [ ]:
gaba_fn = 'gaba_data_2019.txt'
supp_fn = 'supp_data_individual_20170427.txt'

In [ ]:
sdf = utils.load_psychophys(supp_fn)
gaba_col = 'mean_occ_all' #'motor' # or 'occ_binoc', 'mean_occ_all', 'motor'
gdf = utils.load_gaba(gaba_fn, gaba_col)
sdf.head()

In [ ]:
def categorize(df, col, values=None):
    df[col] = df[col].astype("category")
    if values is not None:
        if len(values)==len(df[col].cat.categories):
            df[col].cat.rename_categories(values, inplace=True)
            df[col].cat.reorder_categories(values.values(), inplace=True)
    return df

pop_dict = {'Amblyope':'PWA', 'Control':'NSP'}

gdf = categorize(gdf, 'Population', pop_dict)
sdf = categorize(sdf, 'Population', pop_dict)

In [ ]:
gdf.Population.cat.categories

In [ ]:
demos = pd.read_csv('demos.csv', lineterminator="\r")
# 1 = amb, 0 = control
subs_to_swap_eyes = demos[demos.swapNDE_EY==1].initials.unique() # subjects whose NDE/DE assignment is wrong in sdf
print(subs_to_swap_eyes)
demos

In [ ]:
demos[demos.initials=='nl']

### Set variables used for graphing

In [ ]:
colors2 = {'PWA':'#1f77b4', 'NSP':'#ff7f0e'}
colors_amb = ["#3274a1","#72b4e1"]
colors_con = ["#e1812c", "#ffc68c"]
colors4 = colors_amb + colors_con
traces4 = ['Amblyope-De', 'Amblyope-Nde', 'Control-De', 'Control-Nde']
traces_graph4 = [f"Persons with\nAmblyopia, DE", f"Persons with\nAmblyopia, NDE", \
                 f"Normally-sighted\npersons, DE", f"Normally-sighted\npersons, NDE"]
pal4 = {}
pal4g = {}
for t, c in zip(traces4, colors4):
    pal4[t] = c
for t, c in zip(traces_graph4, colors4):
    pal4g[t] = c
print(pal4, pal4g, sep='\n')
plot_dir = f"plots/cercor2021"

In [ ]:
!ls "plots"

## Demographic stuff from KB

In [ ]:
amb_data = demos[demos['group']==1].copy()
amb_data['motorGABA'] = pd.to_numeric(amb_data['motorGABA'].str.strip(), errors='coerce')
amb_data[["numID","initials","labelGroup","labelNDE","swapNDE_EY","acuityDE","acuityNDE",
          "iadLogMAR","occGABA","motorGABA"]]

In [ ]:
#tt_eyes = st.ttest_ind(amb_data['acuityDE'], amb_data['acuityNDE'], nan_policy='omit') # are the eyes different in acuity?
#print(tt_eyes)
#print("occ gaba vs acuityDE: ", st.spearmanr(amb_data['occGABA'], amb_data['acuityDE'], nan_policy='omit'))
#print("occ gaba vs acuityNDE: ", st.spearmanr(amb_data['occGABA'], amb_data['acuityNDE'], nan_policy='omit'))
print("occ gaba vs IAD: ", st.spearmanr(amb_data['occGABA'], amb_data['iadLogMAR']))
print("motor gaba vs IAD: ", st.spearmanr(amb_data['motorGABA'], amb_data['iadLogMAR'], nan_policy='omit'))

In [ ]:
print("occ gaba vs interocular acuity difference: ", st.pearsonr(amb_data['occGABA'], amb_data['iadLogMAR']))
print("motor gaba vs interocular acuity difference: ", np.corrcoef(amb_data['motorGABA'], amb_data['iadLogMAR'])[0,1])
#print("motor gaba vs interocular acuity difference: ", st.pearsonr(amb_data['motorGABA'], amb_data['iadLogMAR']))

In [ ]:
print(amb_data['occGABA'], amb_data["motorGABA"], sep='\n')

In [ ]:
has_motor = amb_data[~np.isnan(amb_data['motorGABA'])]

In [ ]:
len(amb_data['occGABA']), len(has_motor['motorGABA'])

#### Demographic info for controls

In [ ]:
nsp_demos = demos[demos['group']==0].copy()
nsp_demos

In [ ]:
np.unique(nsp_demos.initials)

In [ ]:
np.count_nonzero(nsp_demos.labelSex=='female')

In [ ]:
demos

### Figure 5

In [ ]:
#with s.PdfPages(f"{plot_dir}/figure_05.pdf") as pdf:
sns.set_context(context="paper", font_scale=1.1)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4), dpi=300)
ax.set_xlim(0.18, 0.23)
ax.set_ylim(0, 1.4)

sns.regplot(data=amb_data, x='occGABA', y='iadLogMAR', color='blue', marker='o', ax=ax, label='Visual cortex', truncate=False)
r, p = st.pearsonr(amb_data['occGABA'], amb_data['iadLogMAR'])
ax.text(.35, 0.88, f"r={r:.2f}, p={p:.3f}", transform=ax.transAxes, fontdict={'color': 'blue'}, horizontalalignment='center')

sns.regplot(data=amb_data, x='motorGABA', y='iadLogMAR', color='grey', marker='x', ax=ax, label='Motor cortex', truncate=False)
r, p = st.pearsonr(has_motor['motorGABA'], has_motor['iadLogMAR'])
ax.text(.35, 0.82, f"r={r:.2f}, p={p:.2f}", transform=ax.transAxes, fontdict={'color': 'grey'}, horizontalalignment='center')

ax.legend()
ax.set_xlabel("GABA:Creatine ratio")
ax.set_ylabel("Interocular acuity difference (logMAR)")
fig.savefig(f"{plot_dir}/figure_05.pdf", dpi='figure')
plt.show()
plt.close('all')

In [ ]:
np.corrcoef(has_motor['motorGABA'], has_motor['iadLogMAR'])

In [ ]:
st.pearsonr(has_motor['motorGABA'], has_motor['iadLogMAR'])

## Analyze tasks separately (before subsetting to include common subjects)

In [ ]:
pp_subjs = np.unique(sdf.Subject)
n_pp_subjs = len(pp_subjs)
gaba_subjs = np.unique(gdf.subjName)
n_gaba_subjs = len(gaba_subjs)
print(f"Psychophysics subjects (n={n_pp_subjs}):\n", pp_subjs)
print(f"GABA subjects (n={n_gaba_subjs}):\n", gaba_subjs)

# GABA only analyses

### GABA t-test, CON v AMB

In [ ]:
gdf

#### Remove AM and TT based on discussions with Kelly 12/2019.

In [ ]:
gdf_reduced = gdf[(gdf.subjName != 'am') & (gdf.subjName !='tt')]
pop_group_reduced = gdf_reduced.groupby("Population")
pop_group_reduced.describe()

In [ ]:
#print(*pop_group_reduced['GABA'])
gaba_per_group = [col for col_name, col in pop_group_reduced['GABA']]
(tstat, pval) = st.ttest_ind(*gaba_per_group, nan_policy='omit')
print(tstat, pval)

In [ ]:
pop_group = gdf.groupby("Population")
pop_group.describe(percentiles=[.5])

In [ ]:
gaba_per_group = [col for col_name, col in pop_group['GABA']]
(tstat, pval) = st.ttest_ind(*gaba_per_group, nan_policy='omit')
print(tstat, pval)

**Thus we find no significant difference in GABA levels between CON and AMB regardless of whether AM and TT are included. At this point we may as well proceed with gdf_reduced only.**

### GABA violin plot, all subjects

In [ ]:
gdf = gdf_reduced.copy()
gaba_df_immutable = gdf_reduced.copy()
gaba_df_immutable.groupby("Population").describe() # Shoud be 14 PWA due to AM, TT exclusion

In [ ]:
(np.sum((gaba_df_immutable['GABA']-.203)**2)**.5)/6

In [ ]:
gaba_df_immutable.to_csv('gaba_data.csv')

In [ ]:
0.009443/(15**.5)

#### Figure 4

In [ ]:
#with s.PdfPages(f"{plot_dir}/figure_04.pdf") as pdf:
#with sns.plotting_context(context=None, font_scale=1.3):
sns.set_context(context="paper", font_scale=1.3)
fig = plt.figure(figsize=(8,6))  # create a figure object
ax = fig.add_subplot(1, 1, 1)
ax = sns.violinplot(y='GABA',x='Presentation',hue='Population',data=gaba_df_immutable,split=True,inner='stick',ax=ax,legend=False, palette=colors2)
ax.legend(loc=4)
ax.xaxis.set_visible(False)
ax.set_ylabel('GABA:Creatine ratio')
sns.despine(left=True, bottom=True, right=True)
#ax.set_yticklabels([])
plt.show(ax.figure)
ax.figure.savefig(f"{plot_dir}/figure_04.eps", format="eps")
plt.show()
plt.close(ax.figure)
plt.close('all')

### Note: the subject with the lowest GABA:Cr ('tt', .162) is not in the psychophysics data

# Select one psychophysical task's data #

In [ ]:
task = 'SS' # 'SS'
sdf = sdf[sdf['Task']==task]
df_to_model = sdf.copy(deep=True) # make a deep copy

In [ ]:
df_to_model.head()

In [ ]:
n_pp_subjs_thistask = len(np.unique(df_to_model.Subject))
amb_subjs = np.unique(df_to_model[df_to_model["Population"]=="PWA"]["Subject"])
print(amb_subjs)
n_amb_subjs_thistask = len(amb_subjs)
print(f"There are {n_pp_subjs_thistask} subjects for Task {task}, of which {n_amb_subjs_thistask} are Amblyopes.")

### Verifying baselines based on KB feedback about fig R2

In [ ]:
onecond = df_to_model[(df_to_model['Presentation']=='nMono') & (df_to_model['Orientation']=='Iso')]

In [ ]:
cnde_subs = onecond[onecond['Trace']=='Control-Nde'].Subject.unique()

In [ ]:
cde_subs = onecond[onecond['Trace']=='Control-De'].Subject.unique()

In [ ]:
np.setdiff1d(cnde_subs, cde_subs)

In [ ]:
thresh_noswap = onecond.groupby(['Task','Orientation','Presentation','Population', 'Eye','Trace','Subject'], as_index=False)['BaselineThresh'].mean().dropna()

In [ ]:
thresh_noswap.head()

In [ ]:
eye_counts = thresh_noswap['Subject'].value_counts().reset_index()

In [ ]:
eye_counts

In [ ]:
nounpaired = (eye_counts[eye_counts.Subject==2])['index'].unique()

In [ ]:
nounpaired

In [ ]:
thresh_noswap_nounpaired = thresh_noswap[thresh_noswap['Subject'].isin(nounpaired)]

In [ ]:
thresh_noswap.groupby(['Trace']).mean().reset_index()

In [ ]:
thresh_noswap_nounpaired.groupby(['Trace']).describe()

In [ ]:
sns.swarmplot(data=thresh_noswap_nounpaired, x="Population", y="BaselineThresh", hue="Eye")

### This is where the NDE/DE should be switched based on KB findings (only affects Controls, luckily)

But as the above plot shows, there are many more in Control-Nde than Control-De

In [ ]:
subs_to_swap_eyes_baselineSS = []
for gv, g in thresh_noswap_nounpaired.groupby(['Subject']):
    print(gv)#, g[['Eye','BaselineThresh']])
    nde_thresh = g[g.Eye=='Nde']['BaselineThresh'].iloc[0]
    de_thresh = g[g.Eye=='De']['BaselineThresh'].iloc[0]
    diff = nde_thresh - de_thresh
    is_nsp = np.all(g['Population']=='NSP')
    if is_nsp and diff < 0:
        print("This subject has a lower threshold in Nde than De but is a NSP!")
        subs_to_swap_eyes_baselineSS.append(g.Subject.iloc[0])

In [ ]:
subs_to_swap_eyes_baselineSS

In [ ]:
def swap_eyevars(df, subs):
    #print(df, df['Subject'], len(df), sep='\n')
    if df['Subject'] in subs: # fix here
        print(df['Subject'], "SWAP!")
        if df['Eye'] == "De":
            df['Eye'] = "Nde"
            df['Trace'] = df['Trace'].replace('-De', '-Nde')
        else:
            df['Eye'] = "De"
            df['Trace'] = df['Trace'].replace('-Nde', '-De')
    return df

def fix_eyes(df):
    disp_cols = ['Subject','Eye','Trace','BaselineThresh']
    # Identify which rows are of subjects that should be swapped
    # Changed 6/18/21 to include the possibility of assigning eyes based on baseline surround suppression
    subs_to_swap_eyes = subs_to_swap_eyes_baselineSS # assign NSP eyes purely based on baselineSS, disregarding swapNde_EY etc.
    rows_to_change = df_to_model[df_to_model.Subject.isin(subs_to_swap_eyes)]
    assert(np.all(rows_to_change.Population == 'NSP')) # should only affect controls
    assert(np.all(rows_to_change.Trace.isin(['Control-De','Control-Nde']))) # these should be swapped along w/ Eye
    fixed = df.apply(swap_eyevars, subs=subs_to_swap_eyes, axis=1)
    print(fixed[disp_cols])
    return fixed

df_to_model_fixeyes = fix_eyes(df_to_model)

In [ ]:
df_to_model_fixeyes.groupby("Trace")['BaselineThresh'].describe()

### Toggle which version of the data to use, "fixed" eyes or not

In [ ]:
df_to_model = df_to_model_fixeyes

# Preparation for Modeling

In [ ]:
df_to_model.head() # note the first rows, they will tell if fixed - ai/Nde/6.9 -> ai/De/6.9 

### Begin grouping data into conditions to model Subject's ThreshElev as a function of MaskContrast

In [ ]:
pp_gvars = ['Task','Orientation','Presentation','Population','Subject','Eye','Trace'] # One condition
pp_gvars_base = pp_gvars + ['BaselineThresh']

groups_with_baseline = df_to_model.groupby(pp_gvars_base)

# Check if there are any conditions with only two data points
for gv, gr in groups_with_baseline:
    if len(gr)<=2:
        print(gv, gr)

### BaselineThresh analysis before we exclude bad fits; since this is observed not modeled its ok

In [ ]:
print(pp_gvars_base)
pp_gvars_base_agg = [v for v in pp_gvars if v != 'Subject']

In [ ]:
pp_gvars_base_agg

In [ ]:
for gv, g in df_to_model.groupby(pp_gvars_base_agg):
    print(gv, len(np.unique(g['BaselineThresh'])))

In [ ]:
thresh_swap = df_to_model.groupby(['Task', 'Orientation', 'Presentation', 'Population', 'Subject','Eye', 'Trace'])['BaselineThresh'].mean().reset_index()
thresh_swap = thresh_swap[(thresh_swap.Orientation=="Iso")&(thresh_swap.Presentation=='nDicho')]
thresh_swap.head()

In [ ]:
eye_counts_swap = thresh_swap.Subject.value_counts()
subs_no_unpaired = eye_counts[eye_counts_swap.reset_index()['Subject']==2]['index'].unique()
print(subs_no_unpaired, len(subs_no_unpaired))

In [ ]:
thresh_swap_nounpaired = thresh_swap[thresh_swap.Subject.isin(subs_no_unpaired)]

In [ ]:
pd.Series.value_counts(thresh_swap_nounpaired.Subject)

In [ ]:
thresh_swap_nounpaired.groupby(['Trace']).describe()

#### Calculate differences, ratios etc for each subject

In [ ]:
baseline_df_withinsubject = thresh_swap.groupby(#thresh_swap_nounpaired.groupby(
                                        ['Task', 'Orientation', 'Presentation', 'Population',
                                        'Subject']).apply(utils.get_interocular_diff, 'BaselineThresh')
baseline_df_withinsubject.rename(columns={'ValueDiff':'BaselineDiff', 'ValueRatio':'BaselineRatio'}, inplace=True)
baseline_df_withinsubject[(baseline_df_withinsubject.Population=="PWA") & (baseline_df_withinsubject.Eye=="De")]

In [ ]:
ade_threshs = baseline_df_withinsubject[(baseline_df_withinsubject.Population=="PWA") & (baseline_df_withinsubject.Eye=="De")]

In [ ]:
baseline_df_withinsubject_diffs = baseline_df_withinsubject.groupby(['Population','Subject'], as_index=False)[['BaselineDiff','BaselineRatio']].mean()
baseline_df_withinsubject_diffs = categorize(baseline_df_withinsubject_diffs, 'Population', {'PWA':'PWA','NSP':'NSP'})
baseline_df_withinsubject_diffs

###  What if we exclude subject 'ah' who looks like a real outlier?

In [ ]:
adiff = baseline_df_withinsubject_diffs[(baseline_df_withinsubject_diffs.Population=="PWA") &
                                        (baseline_df_withinsubject_diffs.Subject!="ah")].dropna()
cdiff = baseline_df_withinsubject_diffs[baseline_df_withinsubject_diffs.Population=="NSP"].dropna()

In [ ]:
print("diffs: NDE-DE")
print(st.ttest_1samp(adiff['BaselineDiff'], popmean=0), len(adiff['BaselineDiff']))
print(st.ttest_1samp(cdiff['BaselineDiff'], popmean=0), len(cdiff['BaselineDiff']))
print(st.ttest_ind(adiff['BaselineDiff'], cdiff['BaselineDiff']))
print("\nratios: NDE/DE")
print(st.ttest_1samp(adiff['BaselineRatio'], popmean=1))
print(st.ttest_1samp(cdiff['BaselineRatio'], popmean=1))
print(st.ttest_ind(adiff['BaselineRatio'], cdiff['BaselineRatio']))

### Ok based on conversation with Michael, 'ah' is excluded now

In [ ]:
baseline_df_withinsubject = baseline_df_withinsubject[baseline_df_withinsubject.Subject != 'ah']
baseline_df_withinsubject_diffs = baseline_df_withinsubject_diffs[baseline_df_withinsubject_diffs.Subject != 'ah']
df_to_model = df_to_model[df_to_model.Subject!='ah']

### GABA vs baseline contrast threshold, before excluding GABA-less people

In [ ]:
baseline_gaba_plot_df = df_to_model.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])

In [ ]:
simple_bg_df = baseline_gaba_plot_df.groupby(['Task','Orientation','Presentation','Population','Subject','Eye','Trace'])[['GABA','BaselineThresh']].agg(np.mean).reset_index()

In [ ]:
simple_bg_df = simple_bg_df.rename(columns={"BaselineThresh":"value"})
simple_bg_df['measure'] = "BaselineThresh"
simple_bg_df.head()

In [ ]:
len(simple_bg_df.Subject.unique())

In [ ]:
simple_bg_df.Trace = simple_bg_df.Trace.astype('category')
simple_bg_df.Trace.cat.reorder_categories(traces4, inplace=True)
simple_bg_df.Trace.cat.rename_categories(traces_graph4, inplace=True)

In [ ]:
simple_bg_df.groupby(['Task','Orientation','Presentation','Population', 'Eye', 'measure'])['value'].describe()

In [ ]:
simple_bg_df_nonan = simple_bg_df[~np.isnan(simple_bg_df.GABA)]

In [ ]:
simple_bg_df_nonan_onecond = (simple_bg_df_nonan[(simple_bg_df_nonan['Presentation']=='nDicho') & (simple_bg_df_nonan['Orientation']=='Iso')])

In [ ]:
len(simple_bg_df_nonan_onecond.Subject.unique())

In [ ]:
simple_bg_df_nonan_onecond

In [ ]:
# check swapped eyes subs
simple_bg_df_nonan_onecond

### Figure 7

In [ ]:
bg_groups = simple_bg_df_nonan.groupby(['Task','Orientation','Presentation','measure'])
#with s.PdfPages(f"{plot_dir}/figure_07.pdf") as pdf:
i = 0 # only do the first group since they're all the same
for gv, gr in bg_groups:
    if i==0:
        g = s.gaba_vs_psychophys_plot(gv, gr, legend=False,
                    log=True, ylim=(2.5, 16), truncate=False,
                    col="Population", col_order=["PWA", "NSP"], hue="Trace",
                    sharex=False, sharey=False,
                    palette=pal4g,
                    n_boot=n_boot, legend_img=False,
                    markers=['o','^','o','^'],
                    aspect=1)
        #g.fig.suptitle(f"{gv}", fontsize=10, y=0.999)
        g.set_titles("")
        #g.fig.subplots_adjust(left=.08, wspace=.2, right=.99)
        g.fig.savefig(f"{plot_dir}/figure_07.pdf")
        plt.close('all')
        i = 1

#### Figure 6 combined into one panel

In [ ]:
baseline_df_withinsubject_diffs

In [ ]:
baseline_df_withinsubject.Trace = baseline_df_withinsubject.Trace.astype('category')
baseline_df_withinsubject.Trace.cat.reorder_categories(traces4, inplace=True)
baseline_df_withinsubject.Trace.cat.rename_categories(traces_graph4, inplace=True)

In [ ]:
baseline_df_withinsubject.head()

In [ ]:
#with s.PdfPages(f"{plot_dir}/figure_06.pdf") as pdf:
sns.set_context(context="paper", font_scale=1.1)
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))#, gridspec_kw={"width_ratios":[2, 1]})
g = sns.boxplot(data=baseline_df_withinsubject,
                x='Trace',
                y='BaselineThresh',#value',
                ax=ax[0],
                palette=pal4g, fliersize=0)
g = sns.swarmplot(data=baseline_df_withinsubject,
                  x='Trace',
                  y='BaselineThresh',
                  ax=ax[0], color=".25")
g.set_xlabel('')
g.set_ylabel('Baseline contrast discrimination threshold (C%)')
print(baseline_df_withinsubject.groupby(['Population'])['Subject'].nunique())


g2 = sns.boxplot(data=baseline_df_withinsubject_diffs,
                 x='Population',
                 #y='BaselineDiff',
                 y='BaselineRatio',
                 ax=ax[1],
                 fliersize=0, palette=colors2)
g2 = sns.swarmplot(data=baseline_df_withinsubject_diffs,
                   x='Population',
                   #y='BaselineDiff',
                   y='BaselineRatio',
                   ax=ax[1],
                   color=".25")
#g2.legend().remove()
g2.set_xlabel('')
#g2.set_ylabel('Interocular difference (NDE-DE) in baseline\ncontrast discrimination thresholds (C%)')
g2.set_ylabel('Interocular ratio (NDE/DE) of baseline\ncontrast discrimination thresholds')
plt.subplots_adjust(left=.12, right=.95, top=.95, bottom=.05, hspace=.2)
print(baseline_df_withinsubject_diffs.groupby(['Population'])['Subject'].nunique())

fig.savefig(f"{plot_dir}/figure_06.eps")
plt.show()
plt.close('all')

In [ ]:
baseline_df_withinsubject.groupby(['Population','Eye']).describe()

## Plot and model the suppression data (RelMaskContrast vs ThreshElev)

In [ ]:
df_to_model.head()

In [ ]:
np.unique(df_to_model.MaskContrast)

In [ ]:
# #bins = [0, 6, 11, 19, 33, 51, 63, 80, 100] # 17/18, 30/32, 39/45/50, 56/60/62, 74/79, and 99%
# bins = [0, 6, 11, 19, 33, 63, 80, 100] # 17/18, 30/32, 39/45/50/56/60/62, 74/79, and 99%
# #bins = [0, 6, 11, 19, 33, 63, 100]# 17/18, 30/32, 39/45/50/56/60/62, 74/79/99%
# df_to_model['Bin2'] = pd.cut(df_to_model['MaskContrast'], bins=bins, labels=[x+1 for x in range(len(bins)-1)])

In [ ]:
np.unique(df_to_model.RelMaskContrast)

In [ ]:
sns.relplot(data=df_to_model, x="MaskContrast", y="ThreshElev", hue="Trace",
                        row="Orientation",
                        #col="Population", col_order=["PWA", "NSP"],
                        col="Presentation", col_order=["nMono", "nDicho"],
                        palette=pal4, style="Eye", markers=['o','d'],
                        height=4.5, aspect=1.3)\
    .map(plt.axhline, y=1, linestyle='dotted', color='black')\
    .set(ylabel="Threshold Elevation")\
    .set(xlabel='Surround Contrast (C%)')
plt.close('all')

In [ ]:
sns.relplot(data=df_to_model, x="RelMaskContrast", y="ThreshElev", hue="Trace",
                        row="Orientation",
                        #col="Population", col_order=["PWA", "NSP"],
                        col="Presentation", col_order=["nMono", "nDicho"],
                        palette=pal4, style="Eye", markers=['o','d'],
                        height=4.5, aspect=1.3)\
.map(plt.axhline, y=1, linestyle='dotted', color='black')\
.map(plt.axvline, x=2, linestyle='dotted', color='grey')\
.map(plt.axvline, x=5, linestyle='dotted', color='grey')\
.map(plt.axvline, x=10, linestyle='dotted', color='grey')\
.set(ylabel="Threshold Elevation")\
.set(xlabel='Relative surround contrast\n (multiples of baseline)')
plt.close('all')

## Pick critical value and observations closest to that
#### Everything after this must be repeated for different critical values to show robustness

In [ ]:
def closest_obs(df, col, to):
    co = (df.iloc[np.argmin(np.abs(df[col] - to))])
    #print(co, co.name, co.index, sep='\n')
    return co

def annotate(data, field="hue", pos=(.05, .7), **kws):
    #print(field, data[['Eye',field]], kws)
    ax=plt.gca()
    for (i, (gv, g)) in enumerate(data.groupby(field)):
        print(i, gv)
        pos_this=(pos[0], pos[1]+(.05*i))
        annotation = f"{gv} n={len(g)}"
        ax.text(*pos_this, annotation, transform=ax.transAxes)
        


In [ ]:
cv = 5
print(f"**Analysis at RelMaskContrast={cv}**\n")
final_obs = df_to_model.groupby(pp_gvars).apply(closest_obs, 'RelMaskContrast', cv).reset_index(drop=True)
sub_groups = final_obs.groupby(['Task','Orientation','Presentation','Population','Subject'], as_index=False)
xdiffs = sub_groups.apply(utils.get_interocular_diff, "RelMaskContrast").dropna().drop_duplicates(['Task','Orientation','Presentation','Subject','ValueDiff'])

#print(xdiffs)
#     sns.catplot(kind='bar', data=xmeans, x='Trace', y='RelMaskContrast', order=traces4, row="Orientation", col="Presentation", palette=pal4)\
#         .set(ylabel='Mean of relative surround contrasts tested')
g1 = sns.catplot(kind='box', data=xdiffs, x='Population', y='ValueDiff', order=["PWA", "NSP"],
            row="Orientation", row_order=["Iso", "Cross"],
            col="Presentation", col_order=["nMono", "nDicho"],
            height=4, aspect=1.1,
            palette=colors2)\
    .map_dataframe(annotate, field='Population', pos=(.7, .1))\
    .set(ylabel=f"Interocular difference (Nde-De) of tested\nrelative surround contrasts at cv={cv}")
g1.fig.subplots_adjust(wspace=.2, hspace=.2)
plt.show()
sns.relplot(data=final_obs, x="RelMaskContrast", y="ThreshElev", hue="Trace",
            row="Orientation", row_order=["Iso", "Cross"],
            col="Presentation", col_order=["nMono", "nDicho"],
            palette=pal4, style="Eye", markers=['o','d'],
            height=4.5, aspect=1.2)\
.map_dataframe(annotate)\
.map(plt.axhline, y=1, linestyle='dotted', color='black')\
.map(plt.axvline, x=cv, linestyle='dotted', color='grey')\
.set(ylabel=f"Threshold Elevation\nat cv={cv}")\
.set(xlabel='Relative surround contrast\n (multiples of baseline)')
plt.show()
plt.close('all')

In [ ]:
final_obs.groupby(['Task','Orientation','Presentation','Population','Eye'])[['RelMaskContrast','ThreshElev']].describe(percentiles=[.5])

In [ ]:
#apply this to final_obs to get t-tests

def test_eye_diffs(g, col):
    ndes = np.unique(g[g.Eye=='Nde'][col])
    des = np.unique(g[g.Eye=='De'][col])
    #g.hist()
    #print("nde mean: ", np.mean(ndes), "de mean: ", np.mean(des))
    res = st.ttest_ind(ndes, des)
    #display(HTML(g.to_html()))
    print(f"({len(ndes)}, {len(des)}), t={res.statistic:.2f}, p={res.pvalue:.2f}")
    return st.ttest_ind(ndes, des)

gs = final_obs.groupby(['Task','Orientation','Presentation','Population'])
for gv, g in gs:
    print(gv)
    test_eye_diffs(g, "ThreshElev")

### Make Figure 3, the modeling example

In [ ]:
onecond = df_to_model[(df_to_model.Orientation=="Iso") & (df_to_model.Presentation=="nDicho")]
ns_onecond = onecond.groupby(['Task','Orientation','Presentation','Population','Eye','Trace'])['Subject'].nunique().reset_index()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(7, 8), dpi=300)

sns.scatterplot(data=onecond, ax=ax1, x="MaskContrast", y="ThreshElev", hue="Trace",
                        palette=pal4, legend=False)
ax1.set(ylabel="Relative contrast\ndiscrimination threshold")
ax1.set(xlabel='Surround Contrast')
ax1.axhline(y=1, linestyle='dotted', color='grey')

sns.scatterplot(data=onecond, ax=ax2, x="RelMaskContrast", y="ThreshElev", hue="Trace",
                        palette=pal4, legend=False)
ax2.set(ylabel="Relative contrast\ndiscrimination threshold")
ax2.set(xlabel='Relative surround contrast\n (multiples of baseline)')
ax2.axhline(y=1, linestyle='dotted', color='grey')
ax2.axvline(x=5, linestyle='dotted', color='black')
print()

final_obs_onecond = onecond.groupby(pp_gvars).apply(closest_obs, 'RelMaskContrast', cv).reset_index(drop=True)

sns.scatterplot(data=final_obs_onecond, ax=ax3, x="RelMaskContrast", y="ThreshElev", hue="Trace",
                        palette=pal4, legend="brief")
ax3.legend_.set_visible(False)
ax3.set(xlim=ax2.get_xlim())
ax3.set(ylabel="Relative contrast\ndiscrimination threshold")
ax3.set(xlabel='Relative surround contrast\n (multiples of baseline)')
ax3.axhline(y=1, linestyle='dotted', color='grey')
ax3.axvline(x=5, linestyle='dotted', color='black')
handles, labels = ax3.get_legend_handles_labels()
h_leg = []
l_leg = []
for (h, l) in zip(handles, labels):
    if l in traces4:
        i = traces4.index(l)
        n = ns_onecond[ns_onecond.Trace==l]['Subject'].iloc[0]
        print(h, l, i, n, traces_graph4[i])
        h_leg.append(h)
        l_leg.append(f"{traces_graph4[i]}\n(N={n})")

fig.legend(h_leg, l_leg, loc='center right')
#fig.legend(pal4g)
fig.subplots_adjust(hspace=.5, right=.7)
plt.savefig(f"{plot_dir}/figure_03.eps")
plt.close('all')

## Subset to include only (GABA and psychophyics) subjects

In [ ]:
gaba_and_pp_subjs = list(np.intersect1d(pp_subjs, gaba_subjs))
n_gaba_and_pp_subjs = len(gaba_and_pp_subjs)

In [ ]:
sdf = sdf[sdf.Subject.isin(gaba_and_pp_subjs)] # only subjects who did _the current_ pp task and GABA
gaba_and_pp_subjs_thistask = np.unique(sdf.Subject)
n_gaba_and_pp_subjs_thistask = len(gaba_and_pp_subjs_thistask)
print(f"Of the {n_gaba_and_pp_subjs} subjects with both GABA and psychophysics data, {n_gaba_and_pp_subjs_thistask} have both for task {task}.\n{gaba_and_pp_subjs_thistask}")

### Remove subjects we don't have data on both GABA/PP for

In [ ]:
gdf = gdf[gdf.subjName.isin(sdf.Subject)] # only subjects who did both tasks
amb_subjs = (gdf[gdf.Population=='PWA'])
nsp_subjs = (gdf[gdf.Population=="NSP"])
print(f'Of the {len(gdf)} subjects with GABA and {task} data, {len(amb_subjs)} are PWA.')
n_this_task = len(gdf)

In [ ]:
final_obs.head()

In [ ]:
comb = final_obs.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])
comb.drop_duplicates(inplace=True)

#subset to include only those subjects with GABA data
comb_gabappsub = comb[~np.isnan(comb['GABA'])]
print(len(comb), len(comb_gabappsub))
comb_gabappsub.Subject.unique()

In [ ]:
comb.head()

In [ ]:
print(pp_gvars)

In [ ]:
baz = pd.melt(comb, id_vars=(pp_gvars + ['RelMaskContrast','BaselineThresh','GABA']),
              value_vars=["ThreshElev"], var_name='measure')
baz.dropna(subset=['GABA'], inplace=True)
baz.Trace = baz.Trace.astype('category')
baz_temp = baz.Trace.copy()
baz.Trace.cat.reorder_categories(traces4, inplace=True)
assert(baz_temp.equals(baz.Trace))
baz.Trace.cat.rename_categories(traces_graph4, inplace=True)

In [ ]:
pal4g

#### Figures 8 and 9 and produced here

In [ ]:
%%time
fig_groups = baz.groupby(['Task', 'Orientation', 'measure'])
for fgv, fg in fig_groups:
    #print(fg)
    assert(fg['Orientation'].nunique()==1) # we're only looking at one surround orientation per figure
    o = fg['Orientation'].iloc[0] # first element can be used for all
    if o=="Cross":
        # figure 8
        outfile = f"{plot_dir}/figure_08.pdf"
        ylim = (0.2,4)
    elif o=="Iso":
        # figure 9
        outfile = f"{plot_dir}/figure_09.pdf"
        ylim = (0.2, 6)
    else:
        # problem
        raise Error
    g = s.gaba_vs_psychophys_plot(fgv, fg, 
                legend_img = False,
                legend = False,
                log = True,
                sharex=False, sharey=False,
                truncate=False,
                ylim = ylim,
                row="Presentation", row_order=["nDicho", "nMono"],
                col="Population", col_order=["PWA","NSP"],
                hue="Trace",
                palette=pal4g,
                annotate=True, boot_func=utils.compare_rs, 
                n_boot=n_boot,
                markers=['o','d','o','d'])#, legend=False)
    g.set_titles("") # top row is dichoptic
    print(g.fig.get_size_inches(), g.fig.get_dpi())
    g.fig.subplots_adjust(left=.1, right=.95, wspace=0.15)
    g.fig.set_dpi(300)
    g.fig.set_size_inches((7,7))
    g.fig.savefig(outfile, dpi='figure')
    plt.close('all')

# Orientation-selective surround suppression analysis
 * 6/7/21 post MAS: made this analysis refer to final_obs, not baz, thus including psychophys subjects we don't have GABA for and necessatitating the removal of GABA - OSSS results

In [ ]:
final_obs.head()

In [ ]:
oss_gvars = ["Task", "Presentation", "Population", "Subject", "Eye", "Trace"]#, "GABA"]
oss_gvars_combeyes = ["Task", "Presentation", "Population", "Subject", "Trace"]#, "GABA"]
#oss_df = baz.groupby(oss_gvars).apply(utils.calculate_orientation_selective_suppression).reset_index().dropna()
oss_df = final_obs.groupby(oss_gvars).apply(utils.calculate_orientation_selective_suppression, col='ThreshElev').reset_index().dropna()
oss_df['measure'] = 'OSSSRatio'

In [ ]:
oss_df.head(n=5)

In [ ]:
oss_df.groupby(['Task', 'Presentation', 'Population','Eye'])['value'].agg(['mean', 'sem'])

In [ ]:
for gv, g in oss_df.groupby(['Task', 'Presentation', 'Population','Eye']):
    print(gv)#, g, sep='\n')
    ttres = st.ttest_1samp(g['value'], popmean=1, nan_policy='omit')
    print(f"N = {len(g)}, mean Iso/Cross ratio: {g['value'].mean():.2f}, t={ttres.statistic:.2f}, p={ttres.pvalue}")
    #print(st.spearmanr(g['GABA'], g['value']))

In [ ]:
# # combine both eyes (omit 'Eye' from grouping vars)
# def oss_mean_combeyes(df, **kwargs):
#     if len(df.Eye.unique())==2:
#         v1 = df[df.Eye=='Nde']['value'].iloc[0]
#         v2 = df[df.Eye=='De']['value'].iloc[0]
#         oss_mean_combeyes = np.mean([v1, v2])
#     elif len(df.Eye.unique())==1:
#         oss_mean_combeyes = df.value.iat[0]
#     else:
#         oss_mean_combeyes = np.nan
#     #print(df)
#     print(f"OSS mean across eyes: {oss_mean_combeyes}")
#     return pd.Series(oss_mean_combeyes, ['value'])
# oss_df_combeyes = oss_df.groupby(oss_gvars_combeyes).apply(oss_mean_combeyes).reset_index().dropna()
# for gv, g in oss_df_combeyes.groupby(['Task', 'Presentation', 'Population']):
#     print(gv, len(np.unique(g['Subject'])), np.unique(g['Subject']))#, g, sep='\n')
#     print(np.mean(g['value']), np.std(g['value']))
#     print(st.ttest_1samp(g['value'], popmean=1, nan_policy='omit'))
#     print(st.spearmanr(g['GABA'], g['value']))

In [ ]:
# for gv, g in oss_df_combeyes.groupby(['Task', 'Presentation']):
#     print(gv, len(np.unique(g['Subject'])), np.unique(g['Subject']))#, g, sep='\n')
#     pwa = g[g.Population=='PWA']
#     nsp = g[g.Population=='NSP']
#     print('NSP: ', st.ttest_1samp(nsp['value'], popmean=1, nan_policy='omit'), 'PWA: ', st.ttest_1samp(pwa['value'], popmean=1, nan_policy='omit'), sep='\n')
#     print('NSP vs PWA 2-samp: ', st.ttest_ind(nsp['value'], pwa['value']))
#     for gv2, g2 in g.groupby(['Population']):
#         print(st.spearmanr(g2['GABA'], g2['value']))